# VNA Instrument
This example notebooks shows how to use the Analysis and Syntheis Polyphase Filter Banks in conjuction to get the frequency response of a system connected between the DAC-ADC. To have some fun, I'm just using the programmable filter that is part of the firmware.

To run this notebook I'm using this configuration:
* DAC_B -> ADC_C (pfbs) DAC_A -> ADC_D
* ADC_C/DAC_A: filter with cascaded PFBs
* ADC_D/DAC_B: VNA instrument.

### Dual DDS and PFBs
For this application, the dual-dds comes in handy. As I said before, the second stage channelizer is implemented by using a DDS per channel and a mixer. The same DDS is fed into the Synthesis PFB, same channel. For a VNA application, what is needed is to inject a single tone into the DUT and read its output. This is easy to achieve in this structure, because the DDS is unique.

The application sets the same mixer frequency for both the ADC and DAC. Then, when a certain frequency is set in one of the channels of the Synthesis PFB, that will excite the exact same channel on the Analysis PFB. The down-conversion will bring the signal on that channel perfectly to DC. Extra decimation can then be used to lower the noise and improve the quality of the measurement.

### Phase matters
It's easy to compute the amplitude response of a system by sweeping the frequency. However, it's not that easy to get the phase response. The design used here, which shares the DDS between both Analysis and Synthesis PFBs, allows to get the phase response of the DUT as a result of the same reference being used on both sides. This demo notebook shows how to calibrate the overall delay of the Analysis/Synthesis structures and then extend the frequency sweep range in order to get a nice phase calibration that may allow to compute delay in the order of ns.

### Phase calibration
Examples cells are provided to run the phase calibration. The process works as follows.
* Coarse delay is estimated for correction. Most of this delay is coming from the digital path.
* Once coarse delay is estimated, the frequency sweep range is increased.
* The slope of phase vs frequency is used to compute fine delay estimation.
* When sweep range is long enough, neighbor PFB channels are crossed and extra phase jumps show up.
* These phase jumps are fixed and can be calibrated out.
Run the cells in order, specially for the phase calibration cells, in order to get correct results.

### Play with it!!!
Once calibration is done, grab a cable and add that into your connection. You should see very clearly that now the phase response has that extra delay added. If the frequency range is large enough (50-80 MHz), you should be able to estimate delay in the order of ns with no problem.

In [ ]:
import sys
sys.path.append('./soft')

from pfbs import *

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftshift

In [ ]:
# Initialize Firmware.
soc = TopSoc('./pfbs_v1.bit')

# Print information.
print(soc)

In [ ]:
#####################################
### Initialize Dual/Filter Chains ###
#####################################

# Dual Chain: it includes both analysis and synthesis chains.
dual = KidsChain(soc, dual=soc['dual'][0])

# Filter chain: includes both analysis and synthesis, cascaded.
filt = FilterChain(soc, chain=soc['filter'][0])

In [ ]:
############################
### Filter Configuration ###
############################
# Set quantization.
filt.analysis.qout(2)
filt.synthesis.qout(1)

# Set mixer.
filt.set_mixer_frequency(1000)

# Enable all channels.
filt.allon()

In [ ]:
############################
### Wide Frequency Sweep ###
############################
# Frequency range.
fstart = 200
fend = 1800

# Quantization.
dual.qout(2)

# Number of pointes per sweep.
N = 100

# Use 80 % the available bandwidth per sweep.
fbw = 0.8*min(dual.analysis.fs,dual.synthesis.fs)

if (fend-fstart)>fbw:
    fstart = np.arange(fstart, fend, fbw)

# Total number  of points.
NT = len(fstart)*N

f_v = np.zeros(NT)
a_v = np.zeros(NT)
phi_v = np.zeros(NT)
for i,ff in enumerate(fstart):
    fend_ = ff+fbw
    if fend_ > fend:
        fend_ = fend
    print("i = {}, fstart = {} MHz, fend = {} MHz.".format(i, ff, fend_))
    
    # Sweep.
    f,a,phi=dual.sweep(ff,fend_,N=N,g=0.1)
    
    # Concat values.
    f_v[i*N:(i+1)*N] = f
    a_v[i*N:(i+1)*N] = a
    phi_v[i*N:(i+1)*N] = phi
    
plt.figure(dpi=150)
plt.plot(f_v,20*np.log10(a_v/max(a_v)))
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");
plt.ylim([-100,10])
plt.savefig('vna_0.jpg')

In [ ]:
#######################
### Frequency Sweep ###
#######################
# Set filter band.
filt.band(flow = 600, fhigh = 601, single=True)

# Quantization.
dual.qout(2)

# Center frequency/bandwidth/points.
fc = 600
df = 50
N = 100

f,a,phi=dual.sweep(fc-df/2,fc+df/2,N=N,g=0.1)

plt.figure(dpi=150)
plt.plot(f,20*np.log10(a/max(a)))
plt.ylim([-100,10])
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");
plt.savefig('vna_1.jpg')

In [ ]:
###############################
### Coarse Delay estimation ###
###############################
# This is the coarse delay estimation. Most of this delay is coming from the digital path in the 4 PFBs.
# To do this coarse estimation, I do a very small frequency sweep and just compute the distance between
# the 2pi jumps of the phase response.
#
# NOTE: I know the delay is in the order of 10-12 us, which allows me to decide what the sweep should be.

# Set filter band.
filt.allon()

# Set mixer.
dual.set_mixer_frequency(250)

# Center frequency/bandwidth/points.
fc = 550
df = 0.5
N = 500

# Frequency Sweep.
f,a,phi = dual.sweep(fc-df/2,fc+df/2,N=N, g=0.1, set_mixer=False)
df, dt = dual.phase_slope(f, phi)
 
print(" ")
print("df = {} MHz, dt = {} us".format(df, dt))

plt.figure(dpi=150)
plt.plot(f,20*np.log10(a/max(a)))
plt.ylim([-100,10])
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");

In [ ]:
##############################
### Phase Correction by DT ###
##############################
# Using the previously computed coarse delay dt, I do phase correction by just removing this linear
# phase component from the phase vs frequency plot.

# DT is in us.
DT = -dt

phi_u, phi_dt = dual.phase_correction(f, phi, DT=DT, phase_cal=0)

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Original Phase");

plt.figure(dpi=150)
plt.plot(f,phi_u)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Unwrap Phase");

plt.figure(dpi=150)
plt.plot(f,phi_dt)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Correctred phase by DT = {:.6f} us".format(DT));

In [ ]:
#############################
### Fine Delay estimation ###
#############################
# To improve the delay estimation, the frequency sweep needs to be enlarged. It is important to sample the phase
# properly, meaning at least 2 times per cycle. For this reason I used the coarse delay and define the resolution.

# Center frequency.
fc = 550

# To sample the phase correctly, I need at least 2 samples every 1/dt
fres = 0.9*1/dt/2

# Delta frequency/points.
df = 5
N = int(np.ceil(df/fres))

# Frequency Sweep.
f,a,phi = dual.sweep(fc-df/2,fc+df/2,N=N, g=0.1, set_mixer=False)
 
plt.figure(dpi=150)
plt.plot(f,20*np.log10(a/max(a)))
plt.ylim([-100,10])
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");

In [ ]:
##############################
### Phase Correction by DT ###
##############################
# The same correction is applied, but over a larger range. Some remaining delay will show up as the phase vs frequency
# response not being flat, and exposing a slope. That extra slope can be added to the first guess dt to improve the
# phase calibration.

# DT is in us.
DT = -dt

phi_u, phi_dt = dual.phase_correction(f, phi, DT=DT, phase_cal=0)

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Original Phase");

plt.figure(dpi=150)
plt.plot(f,phi_u)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Unwrap Phase");

plt.figure(dpi=150)
plt.plot(f,phi_dt)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Correctred phase by DT = {:.6f} us".format(DT));

In [ ]:
####################################
### Jump-based delay computation ###
####################################
# When the frequency sweep is enlarged enough, the boundaries of PFB channels will add extra phase jumps into
# the phase vs frequency response. These jumps are fixed and can be calibrated out. To do this, I use the 
# sweep data and just compute the jumps and average them all. It is ideal to have many jumps to improve the
# computation and have a better correction.

data = dual.phase_fit(f, phi_dt)

# Sampling period (ns).
ts = 1000/dual.analysis.fs

m_avg = 0
for i in range(len(data['fits'])):
    m  = data['fits'][i]['slope']
    x  = data['fits'][i]['data']['x']
    y  = data['fits'][i]['data']['y']
    fn = data['fits'][i]['data']['fn']
    
    m_avg = m_avg + m
      
    print("Slope [{}]\t= {:.5f} ns".format(i,1000*m/(2*np.pi)))
    #plt.figure(dpi=180);
    #plt.plot(x, y, '.', x, fn, '--k');
    #plt.xlabel('Frequency [MHz]')
    #plt.ylabel('Phase [rad]')
    #plt.title('Slope [{}] = {:.5} ns'.format(i,1000*m/(2*np.pi)))
    #plt.savefig('phase-slope-jump-{}.jpg'.format(i))
    
m_avg = m_avg/len(data['fits'])
print("Average Slope\t= {:.5f} ns".format(1000*m_avg/(2*np.pi)))
print(" ")
    
for i in range(len(data['jump']['value'])):
    jv = data['jump']['value'][i]
    
    print("Jump [{}]\t= {:.5f} rad, {:.5f} ns, {:5f} samples".format(i, jv, 1000*jv/(2*np.pi), 1000*jv/(2*np.pi*ts)))
    
j_avg = np.mean(data['jump']['value'])
print("Average Jump\t= {:.5f} rad, {:.5f} ns, {:.5f} samples".format(j_avg, 1000*j_avg/(2*np.pi), 1000*j_avg/(2*np.pi*ts)))

In [ ]:
########################
### Delay estimation ###
########################
# I keep enlarging the frequency range to further improve the delay resolution. Using the previously computed jump
# distance on PFB channel boundaries, I apply that correction together with the delay to calibrate the phase 
# response over a broader range.

# Center frequency.
fc = 550

# To sample the phase correctly, I need at least 2 samples every 1/dt
fres = 0.9*1/dt/2

# Delta frequency/points.
df = 50
N = int(np.ceil(df/fres))

# Frequency Sweep.
f,a,phi = dual.sweep(fc-df/2,fc+df/2,N=N, g=0.1, set_mixer=False)
 
plt.figure(dpi=150)
plt.plot(f,20*np.log10(a/max(a)))
plt.ylim([-100,10])
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");

In [ ]:
##################################################
### Phase Correction by DT and PFB phase jumps ###
##################################################
# With a broad frequency being covered, delay estimation improved. Use this cell and the next back and forth to further
# improve the delay estimation by adding/substraction to the initial dt guess.

# DT is in us.
DT = -dt + 0.004

phi_u, phi_dt = dual.phase_correction(f, phi, DT=DT, phase_cal=j_avg)

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Original Phase");

plt.figure(dpi=150)
plt.plot(f,phi_u)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Unwrap Phase");

plt.figure(dpi=150)
plt.plot(f,phi_dt)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Correctred phase by DT = {:.6f} us".format(DT));

In [ ]:
#################################
### Overall delay computation ###
#################################
# Compute residual slope to further improve delay estimation.

data = dual.phase_fit(f, phi_dt, jumps=False)

m = data['fits'][0]['slope']
x = data['fits'][0]['data']['x']
y = data['fits'][0]['data']['y']
fn = data['fits'][0]['data']['fn']
    
print("Slope = = {} us".format(m/(2*np.pi)))
plt.figure(dpi=150);
plt.plot(x, y, '.', x, fn, '--k');
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Slope = {:.5} ns".format(1000*m/(2*np.pi)));